# **ML Final Project**:

### **Group Members:**
- Christopher Johnson (christopher.johnson13@ontariotechu.net)
- 

## **Project Goals & Outline:**
The goal of this project is to be able to use historical UFO sighting data to accurately predict the likelihood of future encounters when given an input date and location.

**Disclaimer:** While we are aware of the unprofessional nature of this project concept, we wanted to choose a topic that would fit the project guidelines while also being entertaining to both read and work on.

### Outline:
1. Data Pre-processing
2. ...

## **Importing Packages:**

In [90]:
# general packages/libraries
import numpy as np
import pandas as pd
from datetime import datetime # used to convert Date_time strings to useable format

# torch
import torch

# torchmetrics
from torchmetrics import Accuracy

# lightning
from lightning.pytorch import LightningModule
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import CSVLogger

## **Data Importing and Preprocessing:**
This section is where the UFO sighting data is imported and processed to fit the model.

In [91]:
# import the UFO sighing data from the CSV using pandas
ufo_data = pd.read_csv('./ufo-sightings-transformed.csv')
ufo_data.head(5)

,Unnamed: 0,Date_time,date_documented,Year,Month,Hour,Season,Country_Code,Country,Region,Locale,latitude,longitude,UFO_shape,length_of_encounter_seconds,Encounter_Duration,Description
0,0,1949-10-10 20:30:00,4/27/2004,1949,10,20,Autumn,USA,United States,Texas,San Marcos,29.883056,-97.941111,Cylinder,2700.0,45 minutes,This event took place in early fall around 194...
1,1,1949-10-10 21:00:00,12/16/2005,1949,10,21,Autumn,USA,United States,Texas,Bexar County,29.384210,-98.581082,Light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...
2,2,1955-10-10 17:00:00,1/21/2008,1955,10,17,Autumn,GBR,United Kingdom,England,Chester,53.200000,-2.916667,Circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...
3,3,1956-10-10 21:00:00,1/17/2004,1956,10,21,Autumn,USA,United States,Texas,Edna,28.978333,-96.645833,Circle,20.0,1/2 hour,My older brother and twin sister were leaving ...
4,4,1960-10-10 20:00:00,1/22/2004,1960,10,20,Autumn,USA,United States,Hawaii,Kaneohe,21.418056,-157.803611,Light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...


In [94]:
# convert Date_time from string to datetime object
ufo_data['Date_time'] = pd.to_datetime(ufo_data['Date_time'])

day_info = [] # holds day information from ufo_data
minute_info = [] # holds minute information from ufo_data

# extracting the day and minute info from the datetime objects
for datetime_obj in ufo_data['Date_time']:
    day_info.append(datetime_obj.date().day)
    minute_info.append(datetime_obj.time().minute)

# packaging the data into a dictionary
parameters = {
    'Year': ufo_data['Year'].values,
    'Month': ufo_data['Month'].values,
    'Day': day_info,
    'Hour': ufo_data['Hour'].values,
    'Minute': minute_info,
    'Latitude': ufo_data['latitude'].values,
    'Longitude': ufo_data['longitude'].values
}

# converting the dictionary to a dataframe for ease of use
parameters = pd.DataFrame(parameters)

parameters.head(5)

(   Year  Month  Day  Hour  Minute   Latitude   Longitude
 0  1949     10   10    20      30  29.883056  -97.941111
 1  1949     10   10    21       0  29.384210  -98.581082
 2  1955     10   10    17       0  53.200000   -2.916667
 3  1956     10   10    21       0  28.978333  -96.645833
 4  1960     10   10    20       0  21.418056 -157.803611,
    Year  Month  Day  Hour  Minute   Latitude   Longitude
 0  1949     10   10    20      30  29.883056  -97.941111
 1  1949     10   10    21       0  29.384210  -98.581082
 2  1955     10   10    17       0  53.200000   -2.916667
 3  1956     10   10    21       0  28.978333  -96.645833
 4  1960     10   10    20       0  21.418056 -157.803611)